# Exponential approximation exploration

This notebook demonstrates the quality of the sum of exponentials approximation implemented in the library.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from kl_decomposition import gauss_legendre_rule, fit_exp_sum

## Helper utilities

In [ ]:
def l2_error(f, g, x, w):
    return np.sqrt(np.sum(w * (f(x) - g(x))**2))

## Target covariance functions

In [ ]:
def cov_exp(d):
    return np.exp(-d)

def cov_matern32(d):
    return (1 + d) * np.exp(-d)

def cov_matern52(d):
    return (1 + d + d**2/3.) * np.exp(-d)

## Fit and error curves

In [ ]:

x, w = gauss_legendre_rule(0.0, 5.0, 200)
funcs = {
    'Exponential': cov_exp,
    'Matern 3/2': cov_matern32,
    'Matern 5/2': cov_matern52,
}
errors = {name: [] for name in funcs}
ns = range(1, 7)
for n in ns:
    for name, f in funcs.items():
        a, b = fit_exp_sum(n, x, w, f, method='de')
        def approx(t, a=a, b=b):
            return np.sum(a[:, None] * np.exp(-b[:, None] * t[None, :]**2), axis=0)
        err = l2_error(f, approx, x, w)
        errors[name].append(err)
plt.figure(figsize=(6,4))
for name, vals in errors.items():
    plt.semilogy(list(ns), vals, label=name)
plt.xlabel('Number of terms')
plt.ylabel('L2 error')
plt.legend()
plt.tight_layout()
